<a href="https://colab.research.google.com/github/MarvNC/cs523/blob/main/s25_midterm1_handout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<center>
<h1>Midterm 1</h1>
</center>

<hr>

Please work solo on this midterm. No outside human help.


### You need to complete Chapter 7 before this midterm


## Set-up

First bring in your library.


In [ ]:
github_name = "MarvNC"
repo_name = "cs523"
source_file = "library.py"
url = f"https://raw.githubusercontent.com/{github_name}/{repo_name}/main/{source_file}"
# !rm $source_file
# !wget $url
# %run -i $source_file
from library import *

## Here is a new dataset

It is a dataset that looks at various medical tests and shows the `Outcome` of getting Diabetes within 5 years. So the `Outcome` column is the label column (similar to `Survived` being the label column for the Titanic dataset).


In [ ]:
url = "https://www.dropbox.com/s/lq431y13g3zc6lr/w22_pima_midterm1.csv?raw=1"
pima_df = pd.read_csv(url)
pima_df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Region,Outcome
0,6,148.0,medium,35.0,NaN,33.6,0.627,50,C,Yes
1,1,85.0,medium,29.0,NaN,26.6,0.351,31,B,No
2,8,183.0,medium,NaN,NaN,23.3,NaN,32,D,Yes
3,1,89.0,medium,23.0,94.0,28.1,0.167,21,B,No
4,0,137.0,low,35.0,168.0,43.1,2.288,33,B,Yes
...,...,...,...,...,...,...,...,...,...,...
763,10,101.0,medium,48.0,180.0,32.9,0.171,63,D,No
764,2,122.0,medium,27.0,NaN,36.8,0.340,27,A,No
765,5,121.0,medium,23.0,112.0,26.2,0.245,30,B,No
766,1,126.0,low,NaN,NaN,30.1,0.349,47,A,Yes


# Break out into features and labels

I did some work for you by changing the label column into binary.


In [ ]:
pima_features = pima_df.drop(columns="Outcome")
labels = pima_df["Outcome"].replace({"Yes": 1, "No": 0}).to_list()
labels[:5]

/tmp/ipykernel_339521/3101284830.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  labels = pima_df['Outcome'].replace({'Yes':1, 'No':0}).to_list()


[1, 0, 1, 0, 1]

# Challenge 1

The `Outcome` column is the label or target column, the one we are trying to predict.

Go ahead and wrangle the feature columns as follows:

1. All the columns carry potentially useful information. Decide how to deal with categorical columns. Use `ce.TargetEncoder` if need it for nominal column, e.g., Region.

2. For the Tukey transformation, focus on numerical (non-categorical) columns and the `outer` fence.

3. Do scaling on numerical columns.

4. Do imputation as usual.

Put things together in a pipeline and use it. Hint: I ended up with 17 steps.


In [17]:
# Show unique values in BloodPressure Column
pima_df["Region"].unique()

array(['C', 'B', 'D', 'A'], dtype=object)

In [30]:
# fill in your steps
pima_transformer = Pipeline(
    steps=[
        # Pregnancies
        ("Scale Pregnancies", CustomRobustTransformer(target_column="Pregnancies")),
        # Glucose
        ("Scale Glucose", CustomRobustTransformer(target_column="Glucose")),
        # Blood Pressure
        (
            "Map Blood Pressure",
            CustomMappingTransformer(
                mapping_column="BloodPressure",
                mapping_dict={"low": 0, "medium": 1, "high": 2},
            ),
        ),
        # Skin Thickness
        (
            "Scale Skin Thickness",
            CustomRobustTransformer(target_column="SkinThickness"),
        ),
        ("Tukey Skin Thickness", CustomTukeyTransformer(target_column="SkinThickness")),
        # Insulin
        ("Scale Insulin", CustomRobustTransformer(target_column="Insulin")),
        (
            "Tukey Insulin",
            CustomTukeyTransformer(target_column="Insulin"),
        ),
        # BMI
        ("Scale BMI", CustomRobustTransformer(target_column="BMI")),
        ("Tukey BMI", CustomTukeyTransformer(target_column="BMI")),
        # Diabetes Pedigree Function
        (
            "Scale Diabetes Pedigree Function",
            CustomRobustTransformer(target_column="DiabetesPedigreeFunction"),
        ),
        (
            "Tukey Diabetes Pedigree Function",
            CustomTukeyTransformer(target_column="DiabetesPedigreeFunction"),
        ),
        # Age
        ("Scale Age", CustomRobustTransformer(target_column="Age")),
        # Region
        ("Target Region", CustomTargetTransformer(col="Region")),
        # Impute
        ("Impute", CustomKNNTransformer(n_neighbors=5)),
    ],
    verbose=True,
)

In [31]:
pima_transformed_df = pima_transformer.fit_transform(pima_features, labels)

[Pipeline]  (step 1 of 14) Processing Scale Pregnancies, total=   0.0s
[Pipeline] .... (step 2 of 14) Processing Scale Glucose, total=   0.0s


[Pipeline]  (step 3 of 14) Processing Map Blood Pressure, total=   0.0s
[Pipeline]  (step 4 of 14) Processing Scale Skin Thickness, total=   0.0s
[Pipeline]  (step 5 of 14) Processing Tukey Skin Thickness, total=   0.0s
[Pipeline] .... (step 6 of 14) Processing Scale Insulin, total=   0.0s
[Pipeline] .... (step 7 of 14) Processing Tukey Insulin, total=   0.0s
[Pipeline] ........ (step 8 of 14) Processing Scale BMI, total=   0.0s
[Pipeline] ........ (step 9 of 14) Processing Tukey BMI, total=   0.0s
[Pipeline]  (step 10 of 14) Processing Scale Diabetes Pedigree Function, total=   0.0s
[Pipeline]  (step 11 of 14) Processing Tukey Diabetes Pedigree Function, total=   0.0s
[Pipeline] ....... (step 12 of 14) Processing Scale Age, total=   0.0s
[Pipeline] ... (step 13 of 14) Processing Target Region, total=   0.0s
[Pipeline] .......... (step 14 of 14

/home/default/_school/cs523/library.py:1117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_[target] = y


## Match my results


In [32]:
pima_transformed_df.describe().T.round(2)

,count,mean,std,min,25%,50%,75%,max
Pregnancies,768.0,0.17,0.69,-0.60,-0.40,0.00,0.60,3.40
Glucose,768.0,0.11,0.73,-1.74,-0.42,0.00,0.55,1.95
BloodPressure,768.0,1.01,0.66,0.00,1.00,1.00,1.00,2.00
SkinThickness,768.0,0.01,0.66,-1.57,-0.43,0.00,0.43,3.50
Insulin,768.0,0.23,0.80,-0.98,-0.33,0.05,0.57,3.57
BMI,768.0,0.01,0.75,-1.55,-0.53,-0.01,0.47,3.47
DiabetesPedigreeFunction,768.0,0.26,0.83,-0.76,-0.33,0.01,0.67,3.67
Age,768.0,0.25,0.69,-0.47,-0.29,0.00,0.71,3.06
Region,768.0,0.35,0.02,0.33,0.33,0.34,0.36,0.37


## My table

| index                    | count  | mean  | std   | min    | 25%    | 50%    | 75%   | max   |
| ------------------------ | ------ | ----- | ----- | ------ | ------ | ------ | ----- | ----- |
| Pregnancies              | 768\.0 | 0\.17 | 0\.69 | -0\.6  | -0\.4  | 0\.0   | 0\.6  | 3\.4  |
| Glucose                  | 768\.0 | 0\.11 | 0\.73 | -1\.74 | -0\.42 | 0\.0   | 0\.55 | 1\.95 |
| BloodPressure            | 768\.0 | 1\.01 | 0\.66 | 0\.0   | 1\.0   | 1\.0   | 1\.0  | 2\.0  |
| SkinThickness            | 768\.0 | 0\.01 | 0\.66 | -1\.57 | -0\.43 | 0\.0   | 0\.43 | 3\.5  |
| Insulin                  | 768\.0 | 0\.23 | 0\.8  | -0\.98 | -0\.33 | 0\.05  | 0\.57 | 3\.57 |
| BMI                      | 768\.0 | 0\.01 | 0\.75 | -1\.55 | -0\.53 | -0\.01 | 0\.47 | 3\.47 |
| DiabetesPedigreeFunction | 768\.0 | 0\.26 | 0\.83 | -0\.76 | -0\.33 | 0\.01  | 0\.67 | 3\.67 |
| Age                      | 768\.0 | 0\.25 | 0\.69 | -0\.47 | -0\.29 | 0\.0   | 0\.71 | 3\.06 |
| Region                   | 768\.0 | 0\.35 | 0\.02 | 0\.33  | 0\.33  | 0\.34  | 0\.36 | 0\.37 |


In [ ]:
pima_transformed_df.round(2).head()

### My results

| index | Pregnancies | Glucose | BloodPressure | SkinThickness | Insulin | BMI    | DiabetesPedigreeFunction | Age    | Region |
| ----- | ----------- | ------- | ------------- | ------------- | ------- | ------ | ------------------------ | ------ | ------ |
| 0     | 0\.6        | 0\.74   | 1\.0          | 0\.43         | 0\.6    | 0\.14  | 0\.67                    | 1\.24  | 0\.36  |
| 1     | -0\.4       | -0\.76  | 1\.0          | 0\.0          | -0\.47  | -0\.63 | -0\.05                   | 0\.12  | 0\.37  |
| 2     | 1\.0        | 1\.57   | 1\.0          | -0\.01        | 0\.57   | -0\.99 | 0\.56                    | 0\.18  | 0\.34  |
| 3     | -0\.4       | -0\.67  | 1\.0          | -0\.43        | -0\.27  | -0\.46 | -0\.53                   | -0\.47 | 0\.37  |
| 4     | -0\.6       | 0\.48   | 0\.0          | 0\.43         | 0\.38   | 1\.19  | 3\.67                    | 0\.24  | 0\.37  |


# Challenge 2

Use your `find_random_state` function to compute the best value to use for splitting your transformed table.


In [ ]:
%%capture
rs, _ = find_random_state(pima_features, labels, pima_transformer)


In [ ]:
rs  # 188

## Caveat: I'd use 188 going forward

Even if you get a different number, if you want to match my results, use 188.


# Challenge 3

Go ahead and do stratified split into train and test.


In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test =

In [ ]:
x_train[:5]

| index | Pregnancies | Glucose | BloodPressure | SkinThickness | Insulin | BMI   | DiabetesPedigreeFunction | Age | Region |
| ----- | ----------- | ------- | ------------- | ------------- | ------- | ----- | ------------------------ | --- | ------ |
| 762   | 9           | 89\.0   | low           | NaN           | NaN     | 22\.5 | 0\.142                   | 33  | B      |
| 743   | 9           | 140\.0  | high          | NaN           | NaN     | 32\.7 | 0\.734                   | 45  | C      |
| 284   | 2           | 108\.0  | medium        | NaN           | NaN     | 27\.0 | 0\.259                   | 52  | B      |
| 47    | 2           | 71\.0   | medium        | 27\.0         | NaN     | 28\.0 | 0\.586                   | 22  | D      |
| 657   | 1           | 120\.0  | medium        | 48\.0         | 200\.0  | 38\.9 | 1\.162                   | 41  | C      |


# Challenge 4

Now run pipeline:

- fit_transform on x_train, y_train
- transform x_test


In [ ]:
pima_train_df =

In [ ]:
pima_test_df =

In [ ]:
pima_train_df.describe().T.round(2)

### `pima_train_df`

| index                    | count  | mean  | std   | min    | 25%    | 50%   | 75%   | max   |
| ------------------------ | ------ | ----- | ----- | ------ | ------ | ----- | ----- | ----- |
| Pregnancies              | 614\.0 | 0\.17 | 0\.7  | -0\.6  | -0\.4  | 0\.0  | 0\.6  | 3\.4  |
| Glucose                  | 614\.0 | 0\.11 | 0\.73 | -1\.74 | -0\.4  | 0\.0  | 0\.6  | 1\.95 |
| BloodPressure            | 614\.0 | 1\.02 | 0\.66 | 0\.0   | 1\.0   | 1\.0  | 1\.0  | 2\.0  |
| SkinThickness            | 614\.0 | 0\.02 | 0\.63 | -1\.47 | -0\.4  | 0\.0  | 0\.43 | 3\.53 |
| Insulin                  | 614\.0 | 0\.24 | 0\.81 | -1\.0  | -0\.32 | 0\.05 | 0\.59 | 3\.57 |
| BMI                      | 614\.0 | 0\.02 | 0\.75 | -1\.56 | -0\.51 | 0\.0  | 0\.48 | 3\.48 |
| DiabetesPedigreeFunction | 614\.0 | 0\.27 | 0\.83 | -0\.73 | -0\.32 | 0\.0  | 0\.69 | 3\.68 |
| Age                      | 614\.0 | 0\.25 | 0\.69 | -0\.47 | -0\.29 | 0\.0  | 0\.71 | 2\.53 |
| Region                   | 614\.0 | 0\.35 | 0\.02 | 0\.33  | 0\.33  | 0\.35 | 0\.35 | 0\.37 |


In [ ]:
pima_test_df.describe().T.round(2)

### `pima_test_df`

| index                    | count  | mean   | std   | min    | 25%    | 50%    | 75%   | max   |
| ------------------------ | ------ | ------ | ----- | ------ | ------ | ------ | ----- | ----- |
| Pregnancies              | 154\.0 | 0\.18  | 0\.65 | -0\.6  | -0\.4  | 0\.0   | 0\.6  | 2\.0  |
| Glucose                  | 154\.0 | 0\.1   | 0\.71 | -1\.17 | -0\.43 | 0\.0   | 0\.49 | 1\.9  |
| BloodPressure            | 154\.0 | 0\.95  | 0\.64 | 0\.0   | 1\.0   | 1\.0   | 1\.0  | 2\.0  |
| SkinThickness            | 154\.0 | -0\.05 | 0\.59 | -1\.47 | -0\.47 | 0\.0   | 0\.33 | 1\.4  |
| Insulin                  | 154\.0 | 0\.24  | 0\.83 | -0\.97 | -0\.35 | 0\.07  | 0\.53 | 3\.57 |
| BMI                      | 154\.0 | -0\.05 | 0\.78 | -1\.54 | -0\.65 | -0\.07 | 0\.41 | 2\.74 |
| DiabetesPedigreeFunction | 154\.0 | 0\.31  | 0\.79 | -0\.67 | -0\.27 | 0\.09  | 0\.61 | 3\.68 |
| Age                      | 154\.0 | 0\.23  | 0\.71 | -0\.47 | -0\.35 | 0\.0   | 0\.65 | 3\.06 |
| Region                   | 154\.0 | 0\.35  | 0\.02 | 0\.33  | 0\.33  | 0\.35  | 0\.35 | 0\.37 |


# Challenge 5

I used the f1score in chapter 7 but did not really define it. I'll ask you to do that here.

I'll start with 2 random binary numpy arrays of 1000 each. I'll pretend one is y (ground truth) and one is yhat (what we predicted).


In [ ]:
np.random.seed(1234)
yhat = np.random.choice([0, 1], size=(1000,))  # predicted
y = np.random.choice([0, 1], size=(1000,))  # actual

In [ ]:
print(yhat[:20])  # [1 1 0 1 0 0 0 1 1 1 1 1 0 0 1 0 0 0 0 0]
print(y[:20])  # [1 1 1 0 1 1 1 1 1 0 0 0 1 1 0 0 1 1 1 1]

### Here is formula for F1

Sorry, the "p" got lopped off **precision**.

<img src='https://www.dropbox.com/s/kk6p6izjztehdsc/Screen%20Shot%202022-10-17%20at%2012.19.14%20PM.png?raw=1' height=200>


## There are 4 cases

When looking at 2 lists of binary numbers, we are interested on how they match up. Let's assume we look at each pair separately. So y[0] and yhat[0], as example. There are 4 cases to consider, and each has a name.

1. The value in yhat[0] is 1 and the value in y[0] is 1. This is called true positive. It is a correct prediction.

2. The value in yhat[0] is 0 and the value in y[0] is 1. This is called false negative. It is an incorrect prediction.

3. The value in yhat[0] is 0 and the value in y[0] is 0. This is called true negative. It is a correct prediction.

4. The value in yhat[0] is 1 and the value in y[0] is 0. This is called false
   positive. It is an incorrect prediction.

We are interested in counting up how many times each occurs when looking item by item in the two lists.

You will need to compute counts of 3 of these cases to define the f1score function: they will be the body of the function.

I'll first ask you to get some counts using our 2 random binary lists.

Please use list comprehensions.


In [ ]:
#compute true positive  Tp count given yhat and y. Please use list comprehension.

tp =
tp  #260 cases found in the lists

In [ ]:
#compute false negative  Fn count given yhat and y. Please use list comprehension.

fn =
fn  #263 cases found in the lists

In [ ]:
#compute false positive  Fp count given yhat and y.  Please use list comprehension.

fp =
fp  #265 cases found in the lists

### You now have enough to compute precision and recall

Look again at the formula above.

And once you have precision and recall, you can compute the F1 score. Cool.


## Problem with zero-divide

It can be the case that one or both of precision and recall have a 0 in their denominator. Please catch this and just return 0 as the function value.


In [ ]:
#your f1score function defined below. Copy list comprehensions and look out for 0 divide

def f1score(y, yhat):



### Test


In [ ]:
f1score(y, yhat)  # 0.4961832061068703

In [ ]:
f1score([0, 0, 0], [1, 1, 1])  # 0

In [ ]:
f1score([1, 1, 1], [0, 0, 0])  # 0

The oracle.


In [ ]:
from sklearn.metrics import f1_score

f1_score(y, yhat)  # 0.4961832061068703